# Rewriting Symbolic Expressions in Bartiq

As quantum algorithms increase in complexity their symbolic resource expressions similarly become more complex. For a state of the art algorithm like double factorization the resource expressions can be almost impossible to parse, due to the sheer number of terms and symbols.

`bartiq` includes a set of utilities for manipulating symbolic expressions – rewriting them to make them simpler and easier to analyze. They're known as **rewriters**. This functionality is contained in the analysis submodule, and backend-specific rewriters can be imported directly.

In [ ]:
from bartiq.analysis import sympy_rewriter

Below, we load up our double factorization compiled routine. Note we aren't assigning any values to our variables, so this will have **symbolic resource expressions**.

In [ ]:
from bartiq import CompiledRoutine, sympy_backend
import yaml

with open("./double_factorization_compiled.yaml") as f:
    df_symbolic = CompiledRoutine.from_qref(yaml.safe_load(f), sympy_backend)

To see all the resource costs for double factorization we can call the `resource_values` property, but since these expressions are particularly nasty we won't for now. 

As an example, consider arbitrary angle rotations on logical qubits. To implement these fault tolerantly they must be decomposed into a sequence of fixed-angle gates, and often these decompositions contain many $T$-gates. As such the number of rotations in an algorithm may be a value we seek to minimize, or optimize.

To see how many arbitrary angle rotations we need to synthesize for double factorization, we can easily get the expression:

In [ ]:
df_symbolic.resource_values["rotations"]

But, it is not easily understood! For a summary of the symbols in this expression, check out the `02_algorithmic_intro` notebook! 

With rewriters we can apply assumptions and make substitutions, hopefully tidying up this expression in the process.

To get started, we load the expression into the rewriter object:

In [ ]:
rotations = sympy_rewriter(df_symbolic.resource_values["rotations"])
rotations

The `rewriter` variable here is actually a `dataclass`, and in an interactive environment it displays a `KaTeX`-friendly expression so we can see the effect of method calls immediately.

## Substitutions

Rewriters support arbitrary substitutions of symbols or expressions, for example:

In [ ]:
# Here is an example of how to use the `substitute` method.
rotations.substitute("N_spatial - 1", "X")

All occurrences of $N_{spatial} - 1$ have been replaced with a new variable, $X$! Note that this change is **not permanent** as we didn't override the `rotations` variable.

Notice that the term $\max(b_{as}, b_{givens}, b_{mas})$ occurs a number of times in the `rotations` expression. 

One simplification we can make is to define a new variable: $$B:=\max(b_{as}, b_{givens}, b_{mas}).$$ 

In the next cell, use the `.substitute` method to update the `rotations` variable with this change.

In [ ]:
# Edit the line below to make the substitution!
rotations = rotations.substitute("max(b_as, b_givens, b_mas)", "B")
rotations

## Assumptions

We know that each of the $b_{x}$ variables are non-zero and positive, and as these are _bits of precision_ they typically take values between 7 and 15. For simplicity we will say each of these is simply greater than 5, and straightaway we can apply that information with the `assume` method:

In [ ]:
rotations = rotations.assume("b_as > 5").assume("b_mas > 5").assume("b_givens > 5")
rotations

We also have to provide an assumption on our `B` symbol - assumptions are not inherited across linked symbols!

In the cell below, add a new assumption: $B>5$.

In [ ]:
# Edit the line below to add your assumption!
rotations = rotations.assume("B>5")
rotations

Since $B$ is the maximum of $b_{as}$, $b_{mas}$ and $b_{givens}$, we can get rid of the `max(0, -B + b_mas)`  and `max(0, -B + b_givens)` terms with more assumptions.

Note that the assumptions are **not** `b_mas <= B` or `b_givens <= B` - defining a relationship between two symbols is not possible in SymPy! For a more complete explanation as to _why_ this is not possible, please see the Rewriter [summary page](../../concepts/rewriters.md#assumptions) in Concepts.

In [ ]:
rotations = rotations.assume("b_mas - B <= 0").assume("b_givens - B <= 0")
rotations

Amazing! By only adding some assumptions, and a single substitution to make the expression easier to read, we were able to rewrite the expression into a **much** more palatable form. The number of rotations we need to synthesize is simply related to which of the bits of precision is biggest, plus one!

There is also no need to constantly redefine the `rewriter` variable either, since the rewriter functionality permits method chaining we could do the whole thing in one cell:

In [ ]:
rotations = sympy_rewriter(df_symbolic.resource_values["rotations"])
rotations = (
    rotations.substitute("max(b_as, b_givens, b_mas)", "B")
    .assume("b_as > 5")
    .assume("b_mas > 5")
    .assume("b_givens > 5")
    .assume("B > 5")
    .assume("b_mas - B <= 0")
    .assume("b_givens - B <= 0")
)
rotations

In [ ]:
## Printing the original expression for comparison!
rotations.original

# Challenges

Here's all the steps we did above to the `rotations` expression:

- **Substitute** $\max\left(b_{as}, b_{givens}, b_{mas}\right)\rightarrow B$
- **Assume** $b_{as}>5$, $b_{givens}>5$, $b_{mas}>5$ and $B>5$
- **Assume** $b_{as} - B \leq 0$ and $b_{givens} - B \leq 0$

### Challenge 1

Given the following descendant routine, apply the following steps to simplify the $T$-gates resource expression:

- **Substitute** $\max\left(b_{as}, b_{givens}, b_{mas}\right)\rightarrow B$
- **Assume** $B>5$


In [ ]:
one_electron_term_select: CompiledRoutine = (
    df_symbolic.children["LCU_compute_0"]
    .children["DoubleFactorizationSelect_compute_0"]
    .children["OneElectronTermBlockEncoding_compute_0"]
    .children["OneElectronTermSelect_compute_0"]
)

It's worthwhile pausing to look at the expression _after_ the substitution but _before_ the assumption, to see why the simplification happens.

In [ ]:
one_electron_tgates = sympy_rewriter(one_electron_term_select.resource_values["t_gates"])
one_electron_tgates

The substitute is not necessary, but it does make things easier! A minimal set of steps would be 
- **Assume** $\max\left(b_{as}, b_{givens}, b_{mas}\right)>5$


### Challenge 2

Show that the overall number of $T$-gates (i.e. the top-level cost) is exactly:

$$T\mathrm{-gates} = 24\min\left(M_r, \lceil \log_2(R)\rceil\right) + 1$$

<div class="alert alert-block alert-info"> 
<b>HINT</b> 

After applying the assumptions and substitution from Challenge 1, you'll be left with an expression that involves a $1\!\mod\left(\cdot\right)$ term, where the $\cdot$ is a complex expression in-and-of-itself! There are two things to know:

1) There is a difference between how SymPy _displays_ a function, and how it stores that function internally. As such, calling 
```python
    tgates.substitute("1*mod(xxx)", "yyy")
``` 
will raise an error. In the SymPy library, `Mod` takes two arguments, so any substitution must be written like: 
```python
    tgates.substitute("mod(1, xxx)", "yyy")
```

2) To avoid writing out extremely long arguments for substitution, it's possible to use **wildcards** to substitute patterns. Prefacing a symbol with `$` will mark it as 'wild' in the `.substitute` method. For example:
```python
    sympy_rewriter("max(0, a) + max(0, b)").substitute("max(0, $x)", "x")  ## <--- Match all `max(0, ...)` expressions!
    >>> a + b
```
Wildcard substitutions allow us to search for patterns in expressions (in this case, `max(0, ...)`). Use this fact to replace the $1\!\mod\left(\cdot\right)$ expression!
</div>

In [ ]:
t_gates = sympy_rewriter(df_symbolic.resource_values["t_gates"])
t_gates

### Challenge 3

As a bonus challenge, and assuming that the variable $\lambda = 1$ everywhere, show that the number of Toffoli gates is
 $$ \mathrm{Toffs} \approx 4\max(b_{as}, b_{mas}, b_{givens})\left(N_{spatial} - 1\right) + 4N_{spatial} + 4\lceil \log_2(R)\rceil + 2\lceil(\log_2(R + 1))\rceil + 8$$

Assuming the function $\lambda = 1$ everywhere is an aggressive assumption - making this expression only valid for a restricted set of parameters.

In [ ]:
toffs = sympy_rewriter(df_symbolic.resource_values["toffs"])
toffs

<div class="alert alert-block alert-info"> 
<b>INFO</b> 

The function `ntz` in the above expression stands for 'number of trailing zeroes', and is defined as number of zeroes before the decimal point in _binary representation_ of the number.

For example: `ntz(2)=ntz(0b10) = 1`, `ntz(5)=ntz(0b101)=0`, `ntz(256)=ntz(0b100000000)=8`. 

In physical systems none of our symbols will be larger than ~1000, and so the lowest value this function can take is `0` and the largest value it can take is `9`. 

# Rewriter Cheat Sheet

We've already seen how to substitute symbols or expressions for another with `substitute`, and to apply assumptions onto symbols and expressions with `assume`. Here we will provide a quick rundown of the most important and most powerful functions of rewriters. 

- `expand()`: Expand all the brackets in the expression.
- `simplify()`: Call the built-in SymPy `simplify` functionality. Use with care!

These two methods, along with `substitute` and `assume`, are the only ones that rewrite an expression. They return a new instance of the rewriter dataclass, and thus allow for method chaining. 

The remaining methods are useful for gathering information about the expression.

- `focus(symbols: str | Iterable[str])`: Return only those terms in the expression that contain certain `symbols`. This only hides the remaining terms, it does not delete them.
- `all_functions_and_arguments()`: Return all functions and their arguments in the expression, including nested functions.
- `list_arguments_of_function(function_name: str)`: Return all the arguments of a given function. If the function takes multiple arguments, they are returned as a tuple in the order they appear.
- `history()`: View a time-ordered list of all instructions applied to this instance of the rewriter.
- `evaluate_expression(assignments: dict[str, int | float], functions_map: dict[str, callable])`: Evaluate the expression for a specific data point.

The full documentation can be seen [here](https://psiq.github.io/bartiq/latest/concepts/rewriters/).